In [ ]:
!pip install python-docx
!pip install pdfminer.six
!pip install requests
!pip install bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 39.2 MB/s eta 0:00:00


In [ ]:
import os
import json
import time
import requests
import hashlib
from bs4 import BeautifulSoup
from docx import Document
from pdfminer.high_level import extract_text
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed

# Configuración
HEADERS = {"User-Agent": "Mozilla/5.0"}
URL_PRINCIPAL = "https://conveniomarco2.mercadopublico.cl/software3/quote_public/requestquote/lists/"
PATH = '/content/drive/MyDrive/Diplomado/WhitePaper/archivos/'
LISTADO_FILE = PATH + "listado_rapido.json"
PROCESADOS_FILE = PATH + "procesados.json"
CACHE_HASH_FILE = PATH + "cache_hash.txt"
DOWNLOAD_DIR = PATH + "descargas"

os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# Logging con timestamp
def log(mensaje):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {mensaje}")

def calcular_hash_tbody(html):
    soup = BeautifulSoup(html, "html.parser")
    tbody = soup.find("table", {"id": "wk_mp_requestedquote_table"}).find("tbody")
    return hashlib.sha256(tbody.encode("utf-8")).hexdigest()

# Fase 1: Scrapeo rápido del listado
def scrape_listado_rapido():
    log("Iniciando scraping rápido de listado...")
    response = requests.get(URL_PRINCIPAL, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", {"id": "wk_mp_requestedquote_table"})
    filas = table.find("tbody").find_all("tr")
    total = len(filas)
    listado = []

    for idx, row in enumerate(filas, 1):
        cols = row.find_all("td")
        id_raw = cols[0].find("span", class_="id").text.strip()
        id_clean = id_raw.replace("ID ", "")
        descripcion = cols[0].find("span", class_="name-quote").text.strip()
        url_detalle = cols[0].find("a")["href"]

        log(f"[Fase 1] Licitación {idx} de {total} ({id_clean}): {descripcion}")

        item = {
            "id": id_clean,
            "descripcion": descripcion,
            "organismo": cols[1].text.strip(),
            "publicacion_inicio": cols[2].find("b").text.strip(),
            "publicacion_fin": cols[2].find_all("b")[1].text.strip(),
            "evaluacion_inicio": cols[3].find("b").text.strip(),
            "evaluacion_fin": cols[3].find_all("b")[1].text.strip(),
            "estado": cols[6].text.strip(),
            "ofertas": cols[5].text.strip(),
            "url_detalle": url_detalle
        }
        listado.append(item)

    with open(LISTADO_FILE, "w", encoding="utf-8") as f:
        json.dump(listado, f, indent=2, ensure_ascii=False)

    log(f"Listado guardado en {LISTADO_FILE} con {len(listado)} registros.")
    return listado

# Funciones auxiliares de extracción
def extraer_texto_docx(filepath):
    doc = Document(filepath)
    return "\n".join(p.text for p in doc.paragraphs)

def extraer_texto_pdf(filepath):
    return extract_text(filepath)

def descargar_y_extraer(id_licitacion, nombre, url):
    extension = nombre.split('.')[-1].lower()
    dir_destino = os.path.join(DOWNLOAD_DIR, id_licitacion)
    os.makedirs(dir_destino, exist_ok=True)
    filepath = os.path.join(dir_destino, nombre)

    if not os.path.exists(filepath):
        r = requests.get(url, headers=HEADERS)
        with open(filepath, "wb") as f:
            f.write(r.content)

    try:
        if extension == "docx":
            texto = extraer_texto_docx(filepath)
        elif extension == "pdf":
            texto = extract_text(filepath)
        else:
            texto = f"[Tipo no soportado: {extension}]"
    except Exception as e:
        texto = f"[Error al extraer texto: {str(e)}]"

    return texto

# Fase 2: Procesamiento de adjuntos
def procesar_una_licitacion(item):
    log(f"[Fase 2] Procesando ({item['id']}): {item['descripcion']}")
    item["adjuntos"] = []

    try:
        detalle_html = requests.get(item["url_detalle"], headers=HEADERS).text
        detalle_soup = BeautifulSoup(detalle_html, "html.parser")
        archivos = detalle_soup.find_all("div", class_="field file")

        for archivo in archivos:
            nombre = archivo.find("span").text.strip()
            enlace = archivo.find("a")["href"]
            log(f"  Adjunto de {item['id']}: {nombre}")
            texto = descargar_y_extraer(item["id"], nombre, enlace)
            item["adjuntos"].append({
                "nombre": nombre,
                "url": enlace,
                "texto": texto
            })

    except Exception as e:
        log(f"  [ERROR] Licitación {item['id']}: {e}")
        item["adjuntos"].append({"error": str(e)})

    return item

def procesar_adjuntos_paralelo(listado, max_workers=4):
    log(f"Iniciando procesamiento paralelo de {len(listado)} licitaciones con {max_workers} threads...")
    resultados = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futuras = [executor.submit(procesar_una_licitacion, item) for item in listado]

        for idx, future in enumerate(as_completed(futuras), 1):
            try:
                item_procesado = future.result()
                resultados.append(item_procesado)
                log(f"Licitación {idx} procesada: {item_procesado['id']}")
            except Exception as e:
                log(f"[ERROR] en ejecución paralela: {e}")

            # Guardar progreso incremental
            with open(PROCESADOS_FILE, "w", encoding="utf-8") as f:
                json.dump(resultados, f, indent=2, ensure_ascii=False)

    log(f"Procesamiento completo. Guardado en {PROCESADOS_FILE}")
    return resultados


# =========================
# Ejecución principal
# =========================

print("Comprobando cambios en la página principal...")
html = requests.get(URL_PRINCIPAL, headers=HEADERS).text
nuevo_hash = calcular_hash_tbody(html)

if os.path.exists(CACHE_HASH_FILE):
    with open(CACHE_HASH_FILE, "r") as f:
        hash_anterior = f.read().strip()
else:
    hash_anterior = ""

if nuevo_hash == hash_anterior and os.path.exists(LISTADO_FILE):
    log("Sin cambios. Cargando listado desde cache local...")
    with open(LISTADO_FILE, "r", encoding="utf-8") as f:
        listado_rapido = json.load(f)
else:
    listado_rapido = scrape_listado_rapido()
    with open(CACHE_HASH_FILE, "w") as f:
        f.write(nuevo_hash)

datos_finales = procesar_adjuntos_paralelo(listado_rapido, max_workers=4)
log(f"Finalizado. Licitaciones procesadas: {len(datos_finales)}")

Comprobando cambios en la página principal...
[2025-05-06 02:21:04] Iniciando scraping rápido de listado...
[2025-05-06 02:21:27] [Fase 1] Licitación 1 de 1130 (5802363-3680ESMW): Servicio de Desarrollo de un Upgrade Tecnológico de Aplicaciones y Base de Datos, para el Sistema informático de Seguimiento Documental (SSD)
[2025-05-06 02:21:27] [Fase 1] Licitación 2 de 1130 (5802363-8861ZDOC): CONTRATACIÓN DEL SERVICIO MANTENCIONES EVOLUTIVAS PARA LA PLATAFORMA DE SHAREPOINT
[2025-05-06 02:21:27] [Fase 1] Licitación 3 de 1130 (5802363-1394EBMD): SERVICIOS DE EVOLUCIÓN E INNOVACIÓN DE PLATAFORMA SIM-FIMU “PROYECTO DE RECEPCIÓN Y PROCESAMIENTO DE ESTADOS FINANCIEROS PARA EL SECTOR MUNICIPAL (SUBDERE -CGR)”” E3103/2025
[2025-05-06 02:21:27] [Fase 1] Licitación 4 de 1130 (5802363-1788VAGU): CONTRATO DE SOFTWARE - DIRECCION FORM. 87
[2025-05-06 02:21:27] [Fase 1] Licitación 5 de 1130 (5802363-5733AVJA): MANTENCIÓN DE PLATAFORMA MOODLE: CENTRO DE APRENDIZAJE DE FOCALIZACIÓN
[2025-05-06 02:21:27

[2025-05-06 02:23:41] [Fase 2] Procesando (5802363-9484GZXR): Mantención del Sistema Alarmas Facultad de Economía y Negocios (FEN)[2025-05-06 02:23:41] Licitación 92 procesada: 5802363-2157OMVL

[2025-05-06 02:23:42]   Adjunto de 5802363-9484GZXR: f-mantenci_n_del_sistema_alarmas_facultad_de_econom_a_y_negocios_fen_.-1743684750.docx
[2025-05-06 02:23:43] [Fase 2] Procesando (5802363-1897ZNMF): Mantención del Sistema CCTV Facultad de Economía y Negocios (FEN)[2025-05-06 02:23:43] Licitación 93 procesada: 5802363-9484GZXR

[2025-05-06 02:23:45]   Adjunto de 5802363-1897ZNMF: documentacion_camaras_hikvision_y_panasonic_-_fen_actualizacion_2023_v1.0_3_.-1743682863.xlsx
[2025-05-06 02:23:45] [Fase 2] Procesando (5802363-2937YRSY): ADQUISICIÓN DE UNA PLATAFORMA DE SOFTWARE PARA LA GESTIÓN DE LA CALIDAD INSTITUCIONAL Y LA ACREDITACIÓN DEL CENTRO DE FORMACIÓN TÉCNICA DE LA REGIÓN DE COQUIMBO
[2025-05-06 02:23:45] Licitación 94 procesada: 5802363-0683ACVF
[2025-05-06 02:23:45]   Adjunto de 5802

[2025-05-06 02:23:47]   Adjunto de 5802363-2937YRSY: anexos_software_de_calidad.-1743622360.docx


[2025-05-06 02:23:47]   Adjunto de 5802363-7134VFWP: sc_02.3-0135_y_tdr_convenio_marco_-_servicio_desarrollo_para_plataforma_fondos_concr.-1743621263.pdf
[2025-05-06 02:23:48] [Fase 2] Procesando (5802363-0735OWUE): Servicio de Mantención y/o Soporte de Sitio Web y Redes Sociales Acciona año 2025.
[2025-05-06 02:23:48] Licitación 96 procesada: 5802363-2937YRSY


[2025-05-06 02:23:50] [Fase 2] Procesando (5802363-5465VVVD): Soporte sobre la plataforma Power BI
[2025-05-06 02:23:50] Licitación 97 procesada: 5802363-3089XYSN
[2025-05-06 02:23:50]   Adjunto de 5802363-0735OWUE: solicitud_y_tdr_plataforma_acciona-1743619180.pdf


...

[2025-05-06 02:40:58] [Fase 2] Procesando (5802363-0810QOPE): Mantenciones sobre las aplicaciones que soportan la Operación IVA mediante la integración con procesos BigData.
[2025-05-06 02:40:59]   Adjunto de 5802363-8411WRDN: dtpr_anex_gis.docx
[2025-05-06 02:40:59] [Fase 2] Procesando (5802363-5389MNFA): Servicio de cloud computing mediante créditos Azure de Microsoft para la Universidad de Talca
[2025-05-06 02:40:59]   Adjunto de 5802363-8411WRDN: solicitud_de_cotizaci_n._servicio_mantenci_n_gis_2.pdf
[2025-05-06 02:41:00]   Adjunto de 5802363-0810QOPE: anexo_2.docx
[2025-05-06 02:41:01]   Adjunto de 5802363-5389MNFA: 20-31016_cotizacion_id_2234535.pdf
[2025-05-06 02:41:01]   Adjunto de 5802363-0810QOPE: operaci_n_iva_mediante_operaci_n_bigdata.pdf
[2025-05-06 02:41:02] [Fase 2] Procesando (5802363-7475QUHH): APP aplicación de actividad física y cognitiva Adultos mayores
[2025-05-06 02:41:02] [Fase 2] Procesando (5802363-6228ALMH): Servicios de valor agregado para la administración 

[2025-05-06 02:42:59] [Fase 2] Procesando (5802363-5894NBFS): Mejoramiento de experiencia usuario e interfaz usuario de Plataforma de Interoperabilidad
[2025-05-06 02:43:00] [Fase 2] Procesando (5802363-3679CYCO): Proyecto de diseño UI/UX y diseño web para el nuevo Portal Único del Beneficiario (PUB)


[2025-05-06 02:43:00] Licitación 754 procesada: 5802363-5626XWRO
[2025-05-06 02:43:00]   Adjunto de 5802363-9876MJQU: cm_proyecto_de_consultoria_de_metodologia_pmi..pdf
[2025-05-06 02:43:01]   Adjunto de 5802363-3679CYCO: form_cotizacion_cm_ui-ux_nuevo_sgp_beneficiariov1.4_09.09.24.docx
[2025-05-06 02:43:01] [Fase 2] Procesando (5802363-0204LVYP): Proyecto de Optimización y Modernización del sistema de Registro y Evaluación Unificada de Facilitadores (REUF)
[2025-05-06 02:43:02] [Fase 2] Procesando (5802363-4170VVPX): Servicio Mantención para Distribución de Planillas y Constitución de Deudas
[2025-05-06 02:43:02] Licitación 755 procesada: 5802363-9043XEYL
[2025-05-06 02:43:03] [Fase 2] Procesando (5802363-4800LVKW): OPTIMIZACION PLATAFORMA DE PUNTOS DE CULTURA MINCAP
[2025-05-06 02:43:03]   Adjunto de 5802363-0204LVYP: anexo_n_1_antecedentes_equipo_de_trabajo_cv_9_.docx
[2025-05-06 02:43:03]   Adjunto de 5802363-4170VVPX: anexo_n_1_requisitos_m_nimos_de_la_oferta_3.docx
[2025-05-06 02

[2025-05-06 02:44:34]   Adjunto de 5802363-6685SDZM: solicitud_de_adquisici_n_cm_litio_2_4.pdf
[2025-05-06 02:44:34]   Adjunto de 5802363-1557FCQO: solicitud_de_adquisici_n_cm_litio_2_2.pdf
[2025-05-06 02:44:34] [Fase 2] Procesando (5802363-2417FPPH): Programadores para desarrollo de aplicaciones informáticas Onsite
[2025-05-06 02:44:35] [Fase 2] Procesando (5802363-6124ZLTO): SERVICIO DE GENERACIÓN DE CONTENIDOS COMUNICACIONALES – 12 MESES
[2025-05-06 02:44:35] Licitación 803 procesada: 5802363-6665ZFGF
[2025-05-06 02:44:36] [Fase 2] Procesando (5802363-4315ZZOH): SERVICIO DE VIRTUALIZACIÓN DE UN CURSO TEMÁTICO Y UNA CÁPSULA AUDIOVISUAL TEMÁTICA 2024 (ciclo 3)
[2025-05-06 02:44:36] [Fase 2] Procesando (5802363-9239ULHZ): SERVICIO DE VIRTUALIZACIÓN DE DOS CURSOS TEMÁTICOS 2024 (ciclo 2)
[2025-05-06 02:44:37] Licitación 804 procesada: 5802363-8362MQZC
[2025-05-06 02:44:37] Licitación 805 procesada: 5802363-6029KOJJ
[2025-05-06 02:44:37] Licitación 806 procesada: 5802363-3808TTED
[2025-0

[2025-05-06 02:44:40] [Fase 2] Procesando (5802363-9426PDEQ): solo prueba (no ofertar)
[2025-05-06 02:44:40] Licitación 809 procesada: 5802363-6315CIHW
[2025-05-06 02:44:40]   Adjunto de 5802363-2692UBJG: pdf_de_prueba_-_propuesta_adicional_8.pdf
[2025-05-06 02:44:41] [Fase 2] Procesando (5802363-4346LSIX): solo prueba (no ofertar)
[2025-05-06 02:44:41] [Fase 2] Procesando (5802363-9785QBCC): Cotización de prueba - No ofertar
[2025-05-06 02:44:41] Licitación 810 procesada: 5802363-8801ZOSA
[2025-05-06 02:44:41] [Fase 2] Procesando (5802363-2740ESGY): solo prueba (no ofertar)
[2025-05-06 02:44:42] [Fase 2] Procesando (5802363-3095BFOK): solo prueba (no ofertar)
[2025-05-06 02:44:42]   Adjunto de 5802363-9785QBCC: prueba_5.pdf
[2025-05-06 02:44:42] [Fase 2] Procesando (5802363-8453XZLO): solo prueba (no ofertar)
[2025-05-06 02:44:42] Licitación 811 procesada: 5802363-8223YLHK
[2025-05-06 02:44:42]   Adjunto de 5802363-2740ESGY: pdf_de_prueba_-_criterio_1_12.pdf
[2025-05-06 02:44:43]   Ad

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar xf spark-3.5.5-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyarrow==15.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 85.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import to_date

PATH = '/content/drive/MyDrive/Diplomado/WhitePaper/archivos/'
# 1. Crear la SparkSession
spark = SparkSession.builder \
    .appName("LicitacionesScraping") \
    .getOrCreate()

# 2. Leer el JSON
#df = spark.read.option("multiline",True).json(PATH + "procesados.json")

#df.write.mode("overwrite").parquet(PATH + "datos_parquet/")
df = spark.read.parquet()

# 3. Explorar la estructura
df.printSchema()

# 4. Aplanar los adjuntos para que cada fila sea un archivoPATH + "datos_parquet/"
df_flat = df.select(
    "id", "publicacion_inicio", "descripcion", "organismo",
    explode("adjuntos").alias("adjunto")
).select(
    "id", "publicacion_inicio", "descripcion", "organismo",
    "adjunto.nombre", "adjunto.url", "adjunto.texto"
)

# Convertir columna string a fecha real
df_flat = df_flat.withColumn(
    "fecha_publicacion", to_date("publicacion_inicio", "dd-MM-yyyy")
)

# 5. Crear vista SQL
df_flat.createOrReplaceTempView("archivos")

# 6. Consulta SQL
resultados = spark.sql("""
    SELECT id, fecha_publicacion, nombre, LENGTH(texto) AS largo_texto
    FROM archivos
    WHERE texto IS NOT NULL AND texto LIKE '%documental%'
ORDER BY fecha_publicacion Desc
""")

resultados.show(truncate=False)

resultados = spark.sql("""
    SELECT id, fecha_publicacion, nombre, LENGTH(texto) AS largo_texto, texto
    FROM archivos
    WHERE texto IS NOT NULL AND texto LIKE '%documental%'
ORDER BY fecha_publicacion Desc
""")

root
 |-- adjuntos: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- nombre: string (nullable = true)
 |    |    |-- texto: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |-- descripcion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- evaluacion_fin: string (nullable = true)
 |-- evaluacion_inicio: string (nullable = true)
 |-- id: string (nullable = true)
 |-- ofertas: string (nullable = true)
 |-- organismo: string (nullable = true)
 |-- publicacion_fin: string (nullable = true)
 |-- publicacion_inicio: string (nullable = true)
 |-- url_detalle: string (nullable = true)

+----------------+-----------------+-----------------------------------------------------------------------------------------------+-----------+
|id              |fecha_publicacion|nombre                                                                                         |largo_texto|
+----------------+-----------------+-----------

In [ ]:
!pip install openai
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.7 MB/s eta 0:00:00


Ejemplo de resumenes

In [ ]:
import openai
import tiktoken
import textwrap
import re

PATH = "/content/drive/MyDrive/Diplomado/WhitePaper/archivos/"
with open(PATH + "/openai_key.txt") as f:
    openai.api_key = f.read().strip()

client = openai.OpenAI(api_key=openai.api_key)

def limpiar_formato_markdown(texto):
    # Elimina negritas en formato **texto**
    texto = re.sub(r'\*\*(.*?)\*\*', r'\1', texto)
    # Elimina negritas/cursivas en formato __texto__
    texto = re.sub(r'__(.*?)__', r'\1', texto)
    # Elimina cursivas en formato *texto*
    texto = re.sub(r'\*(.*?)\*', r'\1', texto)
    # Elimina cursivas en formato _texto_
    texto = re.sub(r'_(.*?)_', r'\1', texto)
    return texto

def resumir_con_openai(texto, modelo="gpt-4o", max_tokens=500):
    prompt = f"Resume brevemente el siguiente texto, destacando los puntos principales. Utiliza texto plano, sin emojis ni formato markdown:\n\n{texto}"
    chat_models = {"gpt-4o", "gpt-4", "gpt-4-turbo", "gpt-3.5-turbo", "gpt-4o-mini"}

    try:
        if modelo in chat_models:
            response = client.chat.completions.create(
                model=modelo,
                messages=[
                    {"role": "system", "content": "Eres un asistente experto en licitaciones públicas."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=max_tokens
            )
            return response.choices[0].message.content.strip()
        else:
            print("PROMPT: ",prompt[:500])
            response = client.chat.completions.create(
                model=modelo,
                messages=[
                    {"role": "system", "content": "Eres un asistente experto en licitaciones públicas."},
                    {"role": "user", "content": prompt}
                ],
                #temperature=0.3,
                max_completion_tokens=max_tokens,
                reasoning_effort="medium",
                response_format={"type": "text"},
                store=False
            )
            return response.choices[0].message

    except openai.APIError as e:
        return f"[API ERROR] {str(e)}"
    except Exception as e:
        return f"[ERROR] {str(e)}"

In [ ]:
!pip show openai

Name: openai
Version: 1.76.2
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 


In [ ]:
print("::::: MODELO GPT-4o :::::")
resultados_local = resultados.select("id", "nombre", "texto").limit(3).collect()

for fila in resultados_local:
    texto = fila["texto"]
    resumen = resumir_con_openai(texto, modelo="gpt-4o")
    print("*"*20)
    lineas = resumen.splitlines()
    resumen_pretty = "\n".join(textwrap.fill(linea, width=160) for linea in lineas if linea.strip() != "")
    resumen_pretty = limpiar_formato_markdown(resumen_pretty)
    my_string = f"{fila['id']} - {fila['nombre']}\nResumen: {resumen_pretty}\n"
    #wrapped_string = textwrap.fill(my_string, width=160)
    print(my_string)

::::: MODELO GPT-4o :::::
********************
5802363-8861ZDOC - requerimiento_de_mantencion_2_sharepoint_05052025-1746473583.pdf
Resumen: La Dirección de Compras y Contratación Pública busca contratar servicios de mantención evolutiva para la plataforma SharePoint, asegurando su actualización y
funcionalidad. El proyecto requiere 600 horas anuales de servicio durante 12 meses, con un presupuesto máximo de $30.195.200. El proveedor debe designar un Jefe
de Proyecto para coordinar con la DCCP. Las ofertas deben cumplir con requisitos técnicos mínimos, incluyendo experiencia en SharePoint e integraciones, y
contar con un equipo de trabajo calificado. La evaluación de ofertas será técnica y económica, y se aplicarán multas por incumplimientos. La contratación se
regirá por un acuerdo complementario, con pagos mensuales tras la entrega de informes de servicios. Se requiere una garantía de fiel cumplimiento del 5% del
monto total del acuerdo.

********************
5802363-3680ESMW - formul

In [ ]:
print("::::: MODELO gpt-4o-mini")
resultados_local = resultados.select("id", "nombre", "texto").limit(3).collect()

for fila in resultados_local:
    texto = fila["texto"]
    resumen = resumir_con_openai(texto, modelo="gpt-4o-mini")
    print("*"*20)
    lineas = resumen.splitlines()
    resumen_pretty = "\n".join(textwrap.fill(linea, width=160) for linea in lineas if linea.strip() != "")
    resumen_pretty = limpiar_formato_markdown(resumen_pretty)
    my_string = f"{fila['id']} - {fila['nombre']}\nResumen: {resumen_pretty}\n"
    #wrapped_string = textwrap.fill(my_string, width=160)
    print(my_string)

::::: MODELO gpt-4o-mini
********************
5802363-8861ZDOC - requerimiento_de_mantencion_2_sharepoint_05052025-1746473583.pdf
Resumen: Se requiere la contratación de servicios de mantención evolutiva para la plataforma SharePoint, mediante un convenio marco, con un presupuesto máximo de
$30.195.200. El objetivo es asegurar que la plataforma esté actualizada y operativa, apoyando la transformación digital y la mejora continua de los procesos
internos de la Dirección de Compras y Contratación Pública (DCCP). Se necesitan 600 horas anuales de soporte especializado durante un contrato de 12 meses.
El proveedor deberá designar un Jefe de Proyecto que actuará como interlocutor con la DCCP y será responsable de coordinar actividades y resolver incidentes. Se
establecen plazos específicos para la presentación de cotizaciones, consultas y la apertura de ofertas. Las ofertas se evaluarán en dos etapas: técnica y
económica, siendo necesarias ciertas calificaciones mínimas en experiencia y equ

¿Cuál sería el costo aproximado de generar los resumenes de todos los documentos?

In [ ]:
from pyspark.sql.functions import col, length, sum as sum_, count

# 1. Leer el archivo parquet
df = spark.read.parquet(PATH + "datos_parquet/")
df_flat = df.select(
    "id", "publicacion_inicio", "descripcion", "organismo",
    explode("adjuntos").alias("adjunto")
).select(
    "id", "publicacion_inicio", "descripcion", "organismo",
    "adjunto.nombre", "adjunto.url", "adjunto.texto"
)
# 2. Filtrar registros que tienen texto no nulo
df_validos = df_flat.filter(col("texto").isNotNull())

# 3. Calcular suma total de caracteres y cantidad de textos
resumen = df_validos.select(
    sum_(length(col("texto"))).alias("total_caracteres"),
    count("*").alias("total_documentos")
).collect()[0]

total_chars = resumen["total_caracteres"]
total_docs = resumen["total_documentos"]

print(f"Total documentos con texto: {total_docs}")
print(f"Total caracteres: {total_chars:,}")


Total documentos con texto: 1737
Total caracteres: 32,984,797


In [ ]:
# Conversión estimada: 4 caracteres ≈ 1 token
total_tokens_est = total_chars / 4

# Costo por modelo (por 1000 tokens)
precios = {
    "gpt-4o": 0.005,
    "gpt-4": 0.01,
    "gpt-4o-mini": 0.0005
}

print("\nEstimación de tokens:", int(total_tokens_est))

for modelo, precio in precios.items():
    costo = (total_tokens_est / 1000) * precio
    print(f"→ Modelo {modelo.ljust(12)}: US${costo:.2f}")



Estimación de tokens: 8246199
→ Modelo gpt-4o      : US$41.23
→ Modelo gpt-4       : US$82.46
→ Modelo gpt-4o-mini : US$4.12


In [ ]:
from pyspark.sql.functions import col, explode, struct, collect_list, lit
import pandas as pd
import os

ARCHIVO_ORIGEN = PATH + "datos_parquet"
ARCHIVO_CACHE = PATH + "resumenes_incremental.parquet"
ARCHIVO_SALIDA = PATH + "datos_con_resumenes.parquet"
LOTE = 10
# Leer el original
df = spark.read.parquet(ARCHIVO_ORIGEN)

# Explodear los adjuntos
df_exploded = df.select("id", to_date("publicacion_inicio", "dd-MM-yyyy").alias("fecha_publicacion"), explode("adjuntos").alias("adjunto"))

# Seleccionar campos relevantes
df_min = df_exploded.select(
    "id",
    "fecha_publicacion",
    col("adjunto.nombre").alias("nombre"),
    col("adjunto.texto").alias("texto"),
    col("adjunto.url").alias("url")
).filter(col("texto").isNotNull()).orderBy(col("fecha_publicacion").desc())

# Cargar cache si existe
if os.path.exists(ARCHIVO_CACHE):
    df_cache = spark.read.parquet(ARCHIVO_CACHE).toPandas()
    procesados_set = set(zip(df_cache["id"], df_cache["fecha_publicacion"], df_cache["nombre"]))
else:
    df_cache = pd.DataFrame(columns=["id", "nombre", "texto", "url", "resumen"])
    procesados_set = set()

# Filtrar ya procesados
df_pendientes = df_min.filter(~(struct("id", "nombre").isin([struct(l[0], l[1]) for l in procesados_set])))

# Limitar lote
df_lote = df_pendientes.limit(3)

df_lote.show(truncate=False)

+----------------+-----------------+-------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Convertir a Pandas para procesar
df_lote_pd = df_lote.toPandas()
# Si no hay más pendientes, detener
if df_lote_pd.empty:
    print("No quedan adjuntos pendientes por procesar.")
else:
    # Procesar lote

    def resumir_seguro(texto):
        try:
            return resumir_con_openai(texto, modelo="gpt-4o-mini")  # o gpt-4o
        except Exception as e:
            return f"[ERROR] {e}"

    df_lote_pd["resumen"] = df_lote_pd["texto"].apply(resumir_seguro)

    # Combinar con cache
    df_completo = pd.concat([df_cache, df_lote_pd], ignore_index=True).drop_duplicates(subset=["id", "nombre"])

    # Guardar cache actualizado
    df_completo_spark = spark.createDataFrame(df_completo)
    df_completo_spark.write.mode("overwrite").parquet(ARCHIVO_CACHE)

    print(f"Guardado: {len(df_lote_pd)} nuevos resúmenes. Total acumulado: {len(df_completo)}")

Guardado: 3 nuevos resúmenes. Total acumulado: 3


In [ ]:
df_original = spark.read.parquet(ARCHIVO_ORIGEN)
df_resumenes = spark.read.parquet(ARCHIVO_CACHE)

# Crear struct con resumen incluido
df_resumen_struct = df_resumenes.select(
    "id",
    struct(
        col("nombre"),
        col("url"),
        col("texto"),
        col("resumen")
    ).alias("adjunto_completo")
)

# Agrupar por id → reconstruir array de adjuntos
df_agrupado = df_resumen_struct.groupBy("id").agg(
    collect_list("adjunto_completo").alias("adjuntos")
)

# Unir con el resto de los datos originales
df_sin_adjuntos = df_original.drop("adjuntos")
df_final = df_sin_adjuntos.join(df_agrupado, on="id", how="left")

# Guardar archivo completo final
df_final.write.mode("overwrite").parquet(ARCHIVO_SALIDA)
print(f"Archivo enriquecido guardado como: {ARCHIVO_SALIDA}")


Archivo enriquecido guardado como: /content/drive/MyDrive/Diplomado/WhitePaper/archivos/datos_con_resumenes.parquet


In [5]:
from pyspark.sql.functions import col, explode, struct, collect_list, lit, to_date
from pyspark.sql import SparkSession
import pandas as pd
import os

PATH = "/content/drive/MyDrive/Diplomado/WhitePaper/archivos/"
ARCHIVO_SALIDA = PATH + "datos_con_resumenes.parquet"

spark = SparkSession.builder \
    .appName("LicitacionesScraping") \
    .getOrCreate()

df = spark.read.parquet(ARCHIVO_SALIDA)
print(df.count())

# Explodear los adjuntos
df_exploded = df.select("id", to_date("publicacion_inicio", "dd-MM-yyyy").alias("fecha_publicacion"), explode("adjuntos").alias("adjunto"))

# Seleccionar campos relevantes
df_min = df_exploded.select(
    "id",
    "fecha_publicacion",
    col("adjunto.nombre").alias("nombre"),
    #col("adjunto.texto").alias("texto"),
    col("adjunto.url").alias("url"),
    col("adjunto.resumen").alias("resumen")
).orderBy(col("fecha_publicacion").desc())

# Limitar lote
df_lote = df_min.limit(5)

df_lote.show(truncate=False)

1130
+----------------+-----------------+-------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------